In [1]:
import pandas as pd
import numpy as np
from functions import *
import random
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
#Load models
#REMINDER: We are going to need to use 'eval' to get the models usable
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
print(len(models))


###SELECT MODEL TO OPTIMIZE
model_num = 2

5


In [3]:
models

[['model_0',
  'model_1',
  'model_2',
  'model_3',
  'model_4',
  'model_06_13_20',
  'model_06_20_20',
  'model_test'],
 ["LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,\n                   intercept_scaling=1, l1_ratio=None, max_iter=50000,\n                   multi_class='auto', n_jobs=None, penalty='l2',\n                   random_state=75, solver='lbfgs', tol=5.993308915320019,\n                   verbose=0, warm_start=False)",
  "DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',\n                       max_depth=None, max_features=None, max_leaf_nodes=263,\n                       min_impurity_decrease=0.0, min_impurity_split=None,\n                       min_samples_leaf=1, min_samples_split=2,\n                       min_weight_fraction_leaf=0.0, presort='deprecated',\n                       random_state=75, splitter='best')",
  "RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,\n                     

In [4]:
df = pd.read_csv("../data/ufc-master.csv")

In [5]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
df = df[(df['Winner'] == 'Blue') | (df['Winner'] == 'Red')]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [6]:
#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

In [7]:
#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

In [8]:
#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[200:]
odds_train = odds_df[200:]
label_train = label_df[200:]

df_test = df[:200]
odds_test = odds_df[:200]
label_test = label_df[:200]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

200
200
200
4045
4045
4045


In [9]:
display(df_train)
display(df_test)

,Unnamed: 0,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,...,B_Light Heavyweight_rank,B_Middleweight_rank,B_Welterweight_rank,B_Lightweight_rank,B_Featherweight_rank,B_Bantamweight_rank,B_Flyweight_rank,B_Pound-for-Pound_rank,better_rank,label
200,200,Marc-Andre Barriault,Krzystof Jotko,145,-165,145.000000,60.606061,2019-07-27,"Edmonton, Alberta, Canada",Canada,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,1
201,201,Alexis Davis,Viviane Araujo,165,-190,165.000000,52.631579,2019-07-27,"Edmonton, Alberta, Canada",Canada,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Red,1
202,202,Hakeem Dawodu,Yoshinori Horie,-440,350,22.727273,350.000000,2019-07-27,"Edmonton, Alberta, Canada",Canada,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,0
203,203,Gavin Tucker,Seungwoo Choi,120,-140,120.000000,71.428571,2019-07-27,"Edmonton, Alberta, Canada",Canada,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,0
204,204,Alexandre Pantoja,Deiveson Figueiredo,-140,120,71.428571,120.000000,2019-07-27,"Edmonton, Alberta, Canada",Canada,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,Red,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4240,4240,Duane Ludwig,Darren Elkins,-155,135,64.516129,135.000000,2010-03-21,"Broomfield, Colorado, USA",USA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,1
4241,4241,John Howard,Daniel Roberts,-210,175,47.619048,175.000000,2010-03-21,"Broomfield, Colorado, USA",USA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,0
4242,4242,Brendan Schaub,Chase Gormley,-260,220,38.461538,220.000000,2010-03-21,"Broomfield, Colorado, USA",USA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,0
4243,4243,Mike Pierce,Julio Paulino,-420,335,23.809524,335.000000,2010-03-21,"Broomfield, Colorado, USA",USA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,0


,Unnamed: 0,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,...,B_Light Heavyweight_rank,B_Middleweight_rank,B_Welterweight_rank,B_Lightweight_rank,B_Featherweight_rank,B_Bantamweight_rank,B_Flyweight_rank,B_Pound-for-Pound_rank,better_rank,label
0,0,Curtis Blaydes,Alexander Volkov,-400,325,25.000000,325.000000,2020-06-20,"Las Vegas, Nevada, USA",USA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Red,0
1,1,Josh Emmett,Shane Burgos,120,-140,120.000000,71.428571,2020-06-20,"Las Vegas, Nevada, USA",USA,...,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,Red,0
2,2,Raquel Pennington,Marion Reneau,-160,140,62.500000,140.000000,2020-06-20,"Las Vegas, Nevada, USA",USA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Red,0
3,3,Belal Muhammad,Lyman Good,-130,110,76.923077,110.000000,2020-06-20,"Las Vegas, Nevada, USA",USA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,0
4,4,Jim Miller,Roosevelt Roberts,200,-240,200.000000,41.666667,2020-06-20,"Las Vegas, Nevada, USA",USA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,195,Domingo Pilarte,Felipe Colares,-400,325,25.000000,325.000000,2019-07-20,"San Antonio, Texas, USA",USA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,1
196,196,Max Holloway,Frankie Edgar,-335,275,29.850746,275.000000,2019-07-27,"Edmonton, Alberta, Canada",Canada,...,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,Red,0
197,197,Cris Cyborg,Felicia Spencer,-380,315,26.315789,315.000000,2019-07-27,"Edmonton, Alberta, Canada",Canada,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,0
198,198,Geoff Neal,Niko Price,-335,275,29.850746,275.000000,2019-07-27,"Edmonton, Alberta, Canada",Canada,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,0


In [10]:
#1. Set features
#2. Set Hyperparameters
#3. Set EV
#4. Remove Features

In [11]:
models[1][model_num]

"RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,\n                       criterion='entropy', max_depth=None, max_features=None,\n                       max_leaf_nodes=5, max_samples=None,\n                       min_impurity_decrease=0.0, min_impurity_split=None,\n                       min_samples_leaf=1, min_samples_split=2,\n                       min_weight_fraction_leaf=0.0, n_estimators=5,\n                       n_jobs=-1, oob_score=False, random_state=75, verbose=0,\n                       warm_start=False)"

In [12]:
test_model_name = models[0][model_num]
test_model = eval(models[1][model_num])
test_model_features = eval(models[2][model_num])
test_model_ev = eval(models[3][model_num])

In [13]:
old_test_model = test_model
old_test_model_features = test_model_features
old_test_model_ev = test_model_ev

In [14]:
print(df_test.columns.tolist()) #Keeps us from truncating

['Unnamed: 0', 'R_fighter', 'B_fighter', 'R_odds', 'B_odds', 'R_ev', 'B_ev', 'date', 'location', 'country', 'Winner', 'title_bout', 'weight_class', 'gender', 'no_of_rounds', 'B_current_lose_streak', 'B_current_win_streak', 'B_draw', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT', 'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses', 'B_total_rounds_fought', 'B_total_title_bouts', 'B_win_by_Decision_Majority', 'B_win_by_Decision_Split', 'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission', 'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Stance', 'B_Height_cms', 'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak', 'R_current_win_streak', 'R_draw', 'R_avg_SIG_STR_landed', 'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct', 'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought', 'R_total_title_bouts', 'R_win_by_Decision_Majority', 'R_win_by_Decision_Split', 'R_win_by_Decision_Unanimous', 'R_win_by_KO/TKO', 'R_win_b

In [15]:
#1. set features
my_pos_features = ['R_fighter', 'B_fighter', 'R_odds', 'B_odds', 'R_ev', 'B_ev',
       'location', 'country', 'title_bout', 'weight_class', 'gender',
       'no_of_rounds', 'B_current_lose_streak', 'B_current_win_streak',
       'B_draw', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT',
       'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses',
       'B_total_rounds_fought', 'B_total_title_bouts',
       'B_win_by_Decision_Majority', 'B_win_by_Decision_Split',
       'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission',
       'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Stance', 'B_Height_cms',
       'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak',
       'R_current_win_streak', 'R_draw', 'R_avg_SIG_STR_landed',
       'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct',
       'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought',
       'R_total_title_bouts', 'R_win_by_Decision_Majority',
       'R_win_by_Decision_Split', 'R_win_by_Decision_Unanimous',
       'R_win_by_KO/TKO', 'R_win_by_Submission',
       'R_win_by_TKO_Doctor_Stoppage', 'R_wins', 'R_Stance', 'R_Height_cms',
       'R_Reach_cms', 'R_Weight_lbs', 'R_age', 'B_age', 'lose_streak_dif',
       'win_streak_dif', 'longest_win_streak_dif', 'win_dif', 'loss_dif',
       'total_round_dif', 'total_title_bout_dif', 'ko_dif', 'sub_dif',
       'height_dif', 'reach_dif', 'age_dif', 'sig_str_dif', 'avg_sub_att_dif',
       'avg_td_dif', 'empty_arena', 'B_match_weightclass_rank', 'R_match_weightclass_rank', 
        "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank",
        "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 
        'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 
        'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", 
        "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 
        'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 
        'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 
        'B_Pound-for-Pound_rank', 'better_rank']

In [16]:
print(test_model_name)
print(test_model)
print(test_model_features)
print(test_model_ev)



model_2
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features=None,
                       max_leaf_nodes=5, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=5, n_jobs=-1,
                       oob_score=False, random_state=75, verbose=0,
                       warm_start=False)
[]
25


In [17]:
def save_model():
    score = evaluate_model(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)
    models[0][model_num] =  test_model_name 
    models[1][model_num] = test_model
    models[2][model_num] = test_model_features
    models[3][model_num] = test_model_ev
    models[4][model_num] = score    
    
    with open('models.csv', 'w', newline='') as outfile:
        writer = csv.writer(outfile)
        for row in models:
            print("HI")
            writer.writerow(row)

    outfile.close()    

In [18]:
def print_model():
    print()
    print(test_model_name)
    print(test_model)
    print(test_model_features)
    print(test_model_ev)
    print()

In [ ]:
keep_going = True
while(keep_going):
    #1. Set Features
    #get_best_features(pos_features, m, df, cur_features, labels, odds, scale=False)
    test_model_features = (get_best_features(my_pos_features, test_model, df_train, test_model_features, label_train, odds_train, 
                                             min_ev=test_model_ev))
    print_model()
    save_model()
    #2 Set hyperparameters
    #def tune_hyperparameters(input_model, input_features, input_df, input_labels, odds_input):
    test_model = tune_hyperparameters(test_model, test_model_features, df_train, label_train, odds_train, 
                                      min_ev=test_model_ev)
    print_model()
    save_model()
    #3. Set EV
    #def tune_ev(input_model, input_features, input_df, input_labels, odds_input, verbose=False):
    test_model_ev = tune_ev(test_model, test_model_features, df_train, label_train, odds_train, verbose=False)

    old_test_model_features = test_model_features #This prevents
                                                  #an uneccesary loop
    print_model()
    save_model()
    #4. Remove Features
    #def remove_to_improve(cur_features, m, df, labels, odds, scale=False, min_ev = 0):
    test_model_features = remove_to_improve(test_model_features, test_model, df_train, label_train, odds_train, min_ev = test_model_ev)    
    keep_going = False
    if old_test_model != test_model:
        print("The hyperparameters are different")
        print("OLD:")
        print(old_test_model)
        print("NEW:")
        print(test_model)
        keep_going = True
        old_test_model = test_model
    if old_test_model_features != test_model_features:
        print("The features are different")
        print("OLD:")
        print(old_test_model_features)
        print("NEW:")
        print(test_model_features)
        keep_going = True
        old_test_model_features = test_model_features
    if old_test_model_ev != test_model_ev:
        print("The EV is different")
        print("OLD:")
        print(old_test_model_ev)
        print("NEW:")
        print(test_model_ev)
        keep_going = True
        old_test_model_ev = test_model_ev
    print_model()
    save_model()
    

Current best score is: -100
Feature: R_Women's Flyweight_rank Score: 1493.8877386703475
Feature: R_Middleweight_rank Score: 1745.909090909091
The best feature was R_Middleweight_rank.  It scored 1745.909090909091
Current best score is: 1745.909090909091
Feature: R_fighter Score: 2352.161172161172
Feature: B_current_lose_streak Score: 2491.1471861471864
Feature: B_current_win_streak Score: 3478.9588572347193
The best feature was B_current_win_streak.  It scored 3478.9588572347193
Current best score is: 3478.9588572347193
Feature: country Score: 3546.9588572347193
Feature: no_of_rounds Score: 4151.958857234719
Feature: B_Stance Score: 4317.1198607665665
Feature: R_win_by_KO/TKO Score: 4640.554004105728
The best feature was R_win_by_KO/TKO.  It scored 4640.554004105728
Current best score is: 4640.554004105728
Feature: B_fighter Score: 5096.738109855051
The best feature was B_fighter.  It scored 5096.738109855051
Current best score is: 5096.738109855051
NO IMPROVEMENT
FINAL BEST SCORE: 509

(108, 55)

(108,)

(108, 2)

(3, 55)

...

(3, 55)

(3,)

(3, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,-130,110,0.396818,0.603182,0
1,115,-135,0.646555,0.353445,0
2,-260,240,0.764111,0.235889,0



          Number of matches: 3
          Number of bets: 2
          Number of winning bets: 1
          Number of losing bets: 1
          Number of underdog bets: 2
          Number of underdog wins: 1
          Number of underdog losses: 1
          Number of Favorite bets: 0
          Number of favorite wins: 0
          Number of favorite losses: 0
          Number of even bets: 0
          Number of even wins: 0
          Number of even losses: 0
          Profit: 15.0
          Profit per bet: 7.5
          Profit per match: 5.0
          
          
HI
HI
HI
HI
HI
0 4574.935615960771
1 4674.935615960771
2 4600.069305800344
3 4700.069305800344
4 4768.819305800344
5 4768.819305800344
6 4768.819305800344
7 4768.819305800344
8 4768.819305800344
9 4814.765251746291
10 4914.765251746291
11 4914.765251746291
12 4914.765251746291
13 5014.765251746291
14 4999.76525174629
15 4999.76525174629
16 4999.76525174629
17 5099.76525174629
18 5130.799734504912
19 5130.799734504912
20 5271.976205

In [ ]:
#Get new score:
#def evaluate_model(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels, test_odds, verbose=True):
score = evaluate_model(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)

print(test_model_name) 
print(models[0][model_num])
print()
print(test_model)
print(eval(models[1][model_num]))
print()
print(test_model_features) 
print(eval(models[2][model_num]))
print()
print(test_model_ev)
print(eval(models[3][model_num]))

In [ ]:
models[0][model_num] =  test_model_name 
models[1][model_num] = test_model
models[2][model_num] = test_model_features
models[3][model_num] = test_model_ev
models[4][model_num] = score

In [ ]:
with open('test.csv', 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    for row in models:
        print("HI")
        writer.writerow(row)
    
outfile.close()

In [ ]:
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
    
print(len(data))

In [ ]:
data